# READ ME

#### This notebook is implemeted in a repository in github with input and output folders.
#### It has 2 parts:
##### > Part 1: Pre-defined funtions for each technique.
##### > Part 2: Execution of model pipelines, here users can modify which combination of techniques they want to run. The scores will be printed as a csv in output folders. 

# PART 1: FUNCTIONS

# Import library

In [9]:
#importing libraries
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from boruta import BorutaPy
from BorutaShap import BorutaShap
from sklearn.feature_selection import RFE

from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

from collections import Counter
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, roc_auc_score, log_loss, cohen_kappa_score

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 

# Read df

In [3]:
#this function is to read, transform and join 2 data frame

def read_features():
    path = 'input/secom.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['feature_'+str(x+1) for x in range(len(df.columns))]
    return df


def read_target():
    path = 'input/secom_labels.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['status','timestamp']
    df['timestamp'] = pd.to_datetime(df['timestamp'],dayfirst=True)
    return df


# Remove duplicated columns

In [4]:
#find the duplicated features (columns)
def remove_duplicated_columns(df):
    list_duplicate = []
    to_remove = []
    for i in range(0, len(df.columns)):
        l = []
        for j in range(i+1,len(df.columns)):
            if df.iloc[:,i].equals(df.iloc[:,j]) == True:
                if j not in list_duplicate:
                    l.append(j)
                    to_remove.append('feature_'+str(j+1))
                list_duplicate.append(i)
                list_duplicate.append(j)

    return df.drop(columns=to_remove, axis = 1)

# X = remove_duplicated_columns(X)
# X.shape


# Remove columns with Constant volatility (std=0)

In [5]:
def remove_constant_volatility(df):
    df_EDA= df.describe().T
    df_EDA= df_EDA[df_EDA["std"] == 0]
    df = df.drop(axis=1, columns=df_EDA.index)
    return df

# X = remove_constant_volatility(X)
# X.shape

# Remove columns with high %Missing values

In [6]:
def remove_cols_with_high_pct_null(df, null_threshold):
    list_column_with_pct_null = pd.concat([df.isnull().sum(), df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
    list_column_with_pct_null= list_column_with_pct_null[list_column_with_pct_null["Percentage (%)"] >= null_threshold]
    df = df.drop(axis=1, columns=list_column_with_pct_null.index)
    return df

# X = remove_cols_with_high_pct_null(X, 0.8)
# X.shape

# Split data

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1,stratify=y)

# Outlier treatment

In [8]:
#how = ['NaN', '3s' ,'nothing']
def replace_outlier(df, how):
    for col in df:
        ll_col = df[col].mean() - 3 * df[col].std()
        ul_col = df[col].mean() + 3 * df[col].std()
        if how == 'NaN':
            df[col] = np.where(df[col]>ul_col,np.NaN,np.where(df[col]<ll_col,np.NaN,df[col]))
        elif how == '3s':
            df[col] = np.where(df[col]>ul_col,ul_col,np.where(df[col]<ll_col,ll_col,df[col]))
    return df

# Missing value Imputation

In [9]:
#which_weights = ['distance','uniform']

def impute_null_with_knn(X_train, X_test, which_weights):
    #First scale the data 
    scaler = MinMaxScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns= X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns= X_test.columns)

    knn = KNNImputer(n_neighbors=5, weights=which_weights) #check this neighbors = 5

    X_train = pd.DataFrame(knn.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(knn.transform(X_test), columns=X_test.columns)
    
    X_train = pd.DataFrame(scaler.inverse_transform(X_train), columns= X_train.columns)
    X_test = pd.DataFrame(scaler.inverse_transform(X_test), columns= X_test.columns)
    return X_train, X_test

#X_train = impute_null_with_knn(X_train)

In [10]:
def impute_null_with_mice(X_train, X_test): 
    imp = IterativeImputer(max_iter=5, verbose=0, imputation_order='roman', random_state=0)
    X_train = pd.DataFrame(imp.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(imp.transform(X_test), columns=X_test.columns)
    return X_train, X_test

# Feature Selection

In [11]:
#This is BorutaShap with TENTATIVE features

#list_method=['shap','gini']

def BorutaShap_FS (X, y,method_option) :
    #modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, class_weight='balanced_subsample', max_depth=5, random_state=100)
    modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, max_depth=5, random_state=100)

    # define model for resp. classifier
    modelshap.fit(X,y)
    feature_names = np.array(X.columns)
    # define Boruta Sahp feature selection method
    feature_selector = BorutaShap(model=modelshap,
                              importance_measure=method_option,
                              classification=True)  # find all relevant features
    feature_selector.fit(X,y,n_trials=100,sample = False, verbose = False,random_state=100)  
    #feature_selector.plot(which_features='accepted',figsize=(20,10))
    tentative=X.loc[:,feature_selector.tentative]
    selected=feature_selector.Subset()
    selten=pd.concat([selected,tentative],axis=1)
    # call transform() on X to filter it down to selected features
    return  selten

In [12]:
#RFE

#classifier = ['RF', 'SVM']

def RFE_FS (X, y,classify) :
    scaler = MinMaxScaler()
    X_scaled= pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    feature_names = np.array(X_scaled.columns)
    if classify == 'RF':
    # define random forest classifier
        model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
       
    if classify== 'SVM':
        model = SVC(kernel='linear',C=5)
        #rfe = RFECV(estimator = model,scoring='accuracy')
    # find all relevant features
    model.fit(X_scaled, y)
    rfe = RFE(estimator = model,n_features_to_select = 30)
    rfe.fit(X_scaled,y)

     # zip feature names, ranks, and decisions 
    feature_ranks = list(zip(feature_names, 
                             rfe.ranking_, 
                             rfe.support_))

    final_features_rfe = list()
    indexes = np.where(rfe.ranking_ <= 2)
    for x in np.nditer(indexes):
        final_features_rfe.append(feature_names[x])
    
    
    # unscale the data before return
    X_unscaled=pd.DataFrame(scaler.inverse_transform(X_scaled), columns=X_scaled.columns)
    ff_rfe=pd.DataFrame(X_unscaled.filter(final_features_rfe))
    

 # call transform() on X to filter it down to selected features
    return  ff_rfe

In [13]:
#Boruta function with random forest

def BorutaPy_FS (X, y) :
    feature_names = np.array(X.columns)

    # define random forest classifier
    model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
    model.fit(X, y)
    # define Boruta feature selection method
    
    feature_selector = BorutaPy(model, n_estimators='auto', verbose=0, random_state=100, max_iter=140)

    # find all relevant features
    feature_selector.fit(X.to_numpy(),y)

    # check selected features
    ##--feature_selector.support_

    # check ranking of features
    ##--feature_ranking=feature_selector.ranking_

    # zip feature names, ranks, and decisions 
    # feature_ranks = list(zip(feature_names, 
    #                          feature_selector.ranking_, 
    #                          feature_selector.support_))

    # print the results
    ##--for feat in feature_ranks:
    ##--    print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features = list()
    indexes = np.where(feature_selector.ranking_ <= 2) #change to 2
    for x in np.nditer(indexes):
        final_features.append(feature_names[x])
    ##--print(final_features)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features))

# Multicolinearity treatement

In [14]:
#Remove the highly collinear features from data
def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model 
        to generalize and improves the interpretability of the model.

    Inputs: 
        x: features dataframe
        threshold: features with correlations greater than this value are removed

    Output: 
        dataframe that contains only the non-highly-collinear features
    '''

    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i+1):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)

            # If correlation exceeds the threshold
            if val >= threshold:
                #Print the correlated features and the correlation value
                #print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns=drops)

    return x

#remove_collinear_features(X, 0.7)

# Balancing

In [15]:
def sampling(X_train, y_train, sampler):
    
    #SMOTE
    if sampler == 'SMOTE':
        sampler = SMOTE(random_state=100)    
    
    #ROSE
    if sampler == 'ROSE':
        sampler = RandomOverSampler(random_state=100, shrinkage=1)

    #ADASYN
    if sampler == 'ADASYN':
        sampler = ADASYN(random_state=100)
    

    #SMOTTEENN
    if sampler == 'SMOTEENN' :
        sampler = SMOTEENN(random_state=100)
        
        
    #Random under Sampling
    if sampler == "randomunder":
        sampler = RandomUnderSampler(random_state=100)

    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    #counter = Counter(y_resampled)
    #print(counter)
    
    return X_resampled, y_resampled

# X_train, y_train = sampling(X_train, y_train,'SMOTE')
# X_train.shape

# Model

#### Model: Random Forest & Logistic Regression

In [16]:
which_model = ['RF', 'LR']

def run_model(X_train, y_train, X_test, y_test, which_model):

    if which_model == 'RF':
    # building model before balancing data
        model = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5) ###############
    elif which_model == 'LR':
        model = LogisticRegression(random_state=1)
    
    model.fit(X_train,y_train)


    #For TEST SPLIT
    y_pred= model.predict(X_test)
    cf_matrix = confusion_matrix(y_test, y_pred)
    accuracy= accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred) ##
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    sensitivity = cf_matrix[1][1] / ( cf_matrix[1][1] + cf_matrix[1][0] )
    auc = roc_auc_score(y_test, y_pred)
    type_1_error_FP = cf_matrix[1][0]
    type_2_error_FN = cf_matrix[0][1]
    log_loss_ = log_loss(y_test, y_pred)
    cohen_kappa_score_ = cohen_kappa_score(y_test, y_pred)
    #Note by default 1 is the positive label. Therefore, -1 is negative
    #bad waffe -> 2 line of matrix -> POSITIVE -> data = -1

    #For TRAIN SPLIT
    y_pred_train= model.predict(X_train)
    cf_matrix_train = confusion_matrix(y_train, y_pred_train)
    accuracy_train= accuracy_score(y_train, y_pred_train)
    f1_train = f1_score(y_train, y_pred_train) ##
    precision_train = precision_score(y_train, y_pred_train)
    recall_train = recall_score(y_train, y_pred_train)
    sensitivity_train = cf_matrix_train[1][1] / ( cf_matrix_train[1][1] + cf_matrix_train[1][0] )
    auc_train = roc_auc_score(y_train, y_pred_train)
    type_1_error_FP_train = cf_matrix_train[1][0]
    type_2_error_FN_train = cf_matrix_train[0][1]




    return cf_matrix, accuracy, f1, precision, recall, sensitivity, type_1_error_FP, type_2_error_FN, auc,log_loss_,cohen_kappa_score_, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, sensitivity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train

#run_model(X_train, y_train, X_test, y_test)

#### Model: Deep Neural Network

In [17]:
#NN


def run_model_NN(X_train, y_train, X_test, y_test):
    y_train = y_train.replace(-1, 0) #to_cat cannot work with negative numbers
    y_test = y_test.replace(-1, 0)

    input_dim = X_train.shape[1]

    model = Sequential()
    model.add(Dense(50, activation='relu', input_dim=input_dim))
    model.add(Dense(35, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

    # one hot encode outputs
    y_train_c = to_categorical(y_train)
    y_test_c = to_categorical(y_test)


    model.fit(X_train, y_train_c, epochs=10, batch_size=50, verbose=2)


    #For TEST SPLIT
    y_pred = model.predict(X_test)
    #Converting predictions to label
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))

    #Converting one hot encoded test label to label
    test = list()
    for i in range(len(y_test_c)):
        test.append(np.argmax(y_test_c[i]))


    cf_matrix = confusion_matrix(test, pred)
    accuracy= accuracy_score(test, pred)
    f1 = f1_score(test, pred) ##
    precision = precision_score(test, pred)
    recall = recall_score(test, pred)
    sensitivity = cf_matrix[1][1] / ( cf_matrix[1][1] + cf_matrix[1][0] ) #change from specificity -> sensitivity
    auc = roc_auc_score(test, pred)
    type_1_error_FP = cf_matrix[1][0]
    type_2_error_FN = cf_matrix[0][1]
    log_loss_ = log_loss(test, pred)
    cohen_kappa_score_ = cohen_kappa_score(test, pred)
    #Note by default 1 is the positive label. Therefore, -1 is negative
    #bad waffe -> 2 line of matrix -> POSITIVE -> data = -1


    #For TRAIN SPLIT
    y_pred_train = model.predict(X_train)
    #Converting predictions to label
    pred_train = list()
    for i in range(len(y_pred_train)): 
        pred_train.append(np.argmax(y_pred_train[i]))

    #Converting one hot encoded test label to label
    train_ = list()
    for i in range(len(y_train_c)):
        train_.append(np.argmax(y_train_c[i]))

    y_pred_train= model.predict(X_train)
    cf_matrix_train = confusion_matrix(train_, pred_train)
    accuracy_train= accuracy_score(train_, pred_train)
    f1_train = f1_score(train_, pred_train) ##
    precision_train = precision_score(train_, pred_train)
    recall_train = recall_score(train_, pred_train)
    sensitivity_train = cf_matrix_train[1][1] / ( cf_matrix_train[1][1] + cf_matrix_train[1][0] )
    auc_train = roc_auc_score(train_, pred_train)
    type_1_error_FP_train = cf_matrix_train[1][0]
    type_2_error_FN_train = cf_matrix_train[0][1]


    return cf_matrix, accuracy, f1, precision, recall, sensitivity, type_1_error_FP, type_2_error_FN, auc,log_loss_,cohen_kappa_score_, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, sensitivity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train


# PART 2: EXECUTION

In [21]:
fast_track_cols = X_test.columns
fast_track_cols.shape

(466,)

In [23]:
X = read_features()
y = read_target().iloc[:,0]

removing_coli = 'Yes'
#['Yes', 'No']

result = []
i = 1



#step 1:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify=y)
#change random state??

#-----------

# # step 2:
# X_train = remove_duplicated_columns(X_train)
# #step 3:
# X_train = remove_constant_volatility(X_train)
# #step 4:
# X_train = remove_cols_with_high_pct_null(X_train, 0.8) #this can be in the loop too, may be later
# #step 5: remove the same columns from step 2-4 TRAIN_TEST split
# X_test = X_test.loc[:,X_train.columns]

#------------

X_train = X_train.loc[:, fast_track_cols]
X_test = X_test.loc[:, fast_track_cols]

#------------




#step 6-9
replace_outlier_options = ['NaN'] #nothing at all
#replace_outlier_options = ['3s','NaN','nothing']
impute_null_options = ['MICE']
#impute_null_options = ['knn__distance', 'MICE', 'knn__uniform']
FS_options = ['BoS__shap']
#FS_options = ['BoP','BoS__shap', 'BoS__gini', 'RFE__RF', 'RFE__SVM', 'nothing]
sampling_options = ['SMOTEENN','ADASYN','SMOTE','ROSE'] #TRY OTHER SAMPLING
#sampling_options = ['SMOTE','ROSE','ADASYN','SMOTEENN']
model_options = ['RF']
#model_options = ['LR', 'RF', 'NN']
#try RF=10depth

#next: TRY KNN DISTANCE

for replace_with in replace_outlier_options:
    for knn_weight in impute_null_options:
        for classifier_model in FS_options:
            for sampling_technique in sampling_options:
                for Model in model_options:
                    X_train_temp = X_train
                    X_test_temp = X_test
                    y_train_temp = y_train
                    y_test_temp = y_test

                    combined_technique = replace_with +' & '+ knn_weight +' & '+ classifier_model + ' & ' + removing_coli +' & '+ sampling_technique +' & '+ Model

                    #step 6: oulier treatement (on both TRAIN & TEST split)
                    if replace_with != 'nothing':
                        X_train_temp = replace_outlier(X_train_temp, replace_with)
                        X_test_temp = replace_outlier(X_test_temp, replace_with)
                    
                    #step 7: missing value imputation (on both TRAIN & TEST split)
                    if knn_weight == 'knn__distance' or knn_weight == 'knn__uniform':
                        X_train_temp, X_test_temp = impute_null_with_knn(X_train_temp, X_test_temp, knn_weight[-(len(knn_weight)-5):])
                    elif knn_weight == 'MICE':
                        X_train_temp, X_test_temp = impute_null_with_mice(X_train_temp, X_test_temp)

                    #step 8: feature selection (on both TRAIN & TEST split)
                    if classifier_model !='nothing':
                        if classifier_model == 'BoS__shap' or classifier_model == 'BoS__gini':
                            X_train_temp = BorutaShap_FS(X_train_temp, y_train_temp, classifier_model[-(len(classifier_model)-5):])
                        elif classifier_model == 'RFE__RF' or classifier_model == 'RFE__SVM':
                            X_train_temp = RFE_FS(X_train_temp, y_train_temp, classifier_model[-(len(classifier_model)-5):])
                        elif classifier_model == 'BoP':
                            X_train_temp = BorutaPy_FS(X_train_temp, y_train_temp)
                    

                        #step 9: remove multilinear features
                    if removing_coli=='Yes':
                        print('n_cols BEFORE multicolinearity treatement', X_train_temp.shape[1])
                        X_train_temp = remove_collinear_features(X_train_temp, 0.7)
                        print('n_cols AFTER multicolinearity treatement', X_train_temp.shape[1])

                    #apply the same result for TEST
                    X_test_temp = X_test_temp.loc[:,X_train_temp.columns]

                    #print out datasets for backup
                    X_train_temp.to_csv('sampling_visualization/X_train_temp_BEFORESAMPLING_'+str(i)+combined_technique+'.csv')
                    X_test_temp.to_csv('sampling_visualization/X_test_temp_BEFORESAMPLING_'+str(i)+combined_technique+'.csv')
                    y_train_temp.to_csv('sampling_visualization/y_train_temp_BEFORESAMPLING_'+str(i)+combined_technique+'.csv')
                    y_test_temp.to_csv('sampling_visualization/y_test_temp_BEFORESAMPLING_'+str(i)+combined_technique+'.csv')

                    #step 10: balancing only on TRAIN split
                    X_train_temp, y_train_temp = sampling(X_train_temp, y_train_temp, sampling_technique)

                    #print out datasets for backup
                    X_train_temp.to_csv('sampling_visualization/X_train_temp_AFTERSAMPLING_'+str(i)+combined_technique+'.csv')
                    X_test_temp.to_csv('sampling_visualization/X_test_temp_AFTERSAMPLING_'+str(i)+combined_technique+'.csv')
                    y_train_temp.to_csv('sampling_visualization/y_train_temp_AFTERSAMPLING_'+str(i)+combined_technique+'.csv')
                    y_test_temp.to_csv('sampling_visualization/y_test_temp_AFTERSAMPLING_'+str(i)+combined_technique+'.csv')


                    #step 11: train model, predict, and print scores
                    if Model != 'NN':
                        try:
                            cf_matrix, accuracy, f1, precision, recall, sensitivity, type_1_error_FP, type_2_error_FN, auc, log_loss_,cohen_kappa_score_, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, sensitivity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train = run_model(X_train_temp, y_train_temp, X_test_temp, y_test_temp, Model)
                        except Exception:
                            cf_matrix = accuracy = f1 = precision = recall = sensitivity = type_1_error_FP = type_2_error_FN = auc = log_loss_ =cohen_kappa_score_ = cf_matrix_train = accuracy_train = f1_train = precision_train = recall_train = sensitivity_train = type_1_error_FP_train = type_2_error_FN_train = auc_train = 0
                    elif Model == 'NN':
                        try:
                            cf_matrix, accuracy, f1, precision, recall, sensitivity, type_1_error_FP, type_2_error_FN, auc, log_loss_,cohen_kappa_score_, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, sensitivity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train = run_model_NN(X_train_temp, y_train_temp, X_test_temp, y_test_temp)
                        except Exception:
                            cf_matrix = accuracy = f1 = precision = recall = sensitivity = type_1_error_FP = type_2_error_FN = auc = log_loss_ =cohen_kappa_score_ = cf_matrix_train = accuracy_train = f1_train = precision_train = recall_train = sensitivity_train = type_1_error_FP_train = type_2_error_FN_train = auc_train = 0

                    result.append((i, combined_technique, X_train_temp.shape[1], replace_with, knn_weight, classifier_model, removing_coli, sampling_technique, Model, X_train_temp.columns, cf_matrix, accuracy, f1, precision, recall, sensitivity, type_1_error_FP, type_2_error_FN, auc, log_loss_, cohen_kappa_score_, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, sensitivity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train))
                    
                    print('attemp no.: ',i, combined_technique,' acc: ', accuracy,' accuracy_train: ',accuracy_train, ' f1: ', f1,' f1_train: ',f1_train, ' sensitivity: ', sensitivity, ' :auc ', auc, ' cfm: ', '\n', cf_matrix, '\n n_cols', X_train_temp.shape[1])
                    

                    if i%5==0: 
                        df_result = pd.DataFrame(result, columns = ['No.','combination','n_cols','outlier_replace_with', 'imputation_knn_weight', 'FS_classifier_model', 'removing_coli','balancing_sampling_technique', 'Model','cols','cf_matrix', 'accuracy', 'f1', 'precision', 'recall', 'sensitivity', 'type_1_error_FP', 'type_2_error_FN', 'auc', 'log_loss_','cohen_kappa_score_','cf_matrix_train', 'accuracy_train', 'f1_train', 'precision_train', 'recall_train', 'sensitivity_train', 'type_1_error_FP_train', 'type_2_error_FN_train', 'auc_train'])
                        df_result.to_csv('tracker/result'+str(i)+'.csv')
                    

                    i+=1



df_result = pd.DataFrame(result, columns = ['No.','combination','n_cols','outlier_replace_with', 'imputation_knn_weight', 'FS_classifier_model', 'removing_coli','balancing_sampling_technique', 'Model','cols','cf_matrix', 'accuracy', 'f1', 'precision', 'recall', 'sensitivity', 'type_1_error_FP', 'type_2_error_FN', 'auc', 'log_loss_','cohen_kappa_score_','cf_matrix_train', 'accuracy_train', 'f1_train', 'precision_train', 'recall_train', 'sensitivity_train', 'type_1_error_FP_train', 'type_2_error_FN_train', 'auc_train'])
df_result.to_csv('tracker/result.csv')



100%|██████████| 100/100 [01:16<00:00,  1.31it/s]


n_cols BEFORE multicolinearity treatement 14
n_cols AFTER multicolinearity treatement 11
attemp no.:  1 NaN & MICE & BoS__shap & Yes & SMOTEENN & RF  acc:  0.7038216560509554  accuracy_train:  0.8907710280373832  f1:  0.20512820512820512  f1_train:  0.9097054563013037  sensitivity:  0.5714285714285714  :auc  0.6423695758166748  cfm:  
 [[209  84]
 [  9  12]] 
 n_cols 11


100%|██████████| 100/100 [03:44<00:00,  2.25s/it]


n_cols BEFORE multicolinearity treatement 11
n_cols AFTER multicolinearity treatement 11
attemp no.:  2 NaN & MICE & BoS__shap & Yes & ADASYN & RF  acc:  0.7770700636942676  accuracy_train:  0.8735244519392917  f1:  0.20454545454545453  f1_train:  0.8830865159781762  sensitivity:  0.42857142857142855  :auc  0.6153096050706972  cfm:  
 [[235  58]
 [ 12   9]] 
 n_cols 11


100%|██████████| 100/100 [01:26<00:00,  1.15it/s]


n_cols BEFORE multicolinearity treatement 14
n_cols AFTER multicolinearity treatement 11
attemp no.:  3 NaN & MICE & BoS__shap & Yes & SMOTE & RF  acc:  0.802547770700637  accuracy_train:  0.8423076923076923  f1:  0.20512820512820512  f1_train:  0.8376594808622965  sensitivity:  0.38095238095238093  :auc  0.6068584430359174  cfm:  
 [[244  49]
 [ 13   8]] 
 n_cols 11


100%|██████████| 100/100 [01:25<00:00,  1.17it/s]


n_cols BEFORE multicolinearity treatement 10
n_cols AFTER multicolinearity treatement 9
attemp no.:  4 NaN & MICE & BoS__shap & Yes & ROSE & RF  acc:  0.8089171974522293  accuracy_train:  0.8521367521367521  f1:  0.23076923076923078  f1_train:  0.8502164502164503  sensitivity:  0.42857142857142855  :auc  0.6323744514870795  cfm:  
 [[245  48]
 [ 12   9]] 
 n_cols 9
